In [1]:
import plotly.graph_objs as go
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
df=pd.read_csv('World.csv')
building=df[(df['Energy Sector']=='Buildings/Others') & (df['Carrier Type']=='Electricity') & (df['Year']==2020)]

In [4]:
app = JupyterDash(__name__)

app.layout = html.Div([
    html.Div([
        dcc.Graph(id = "graph2")
        ]),
    html.Div([
        html.Label("Graphs"), 
        dcc.Dropdown(
            id = "w_dropdown",
            options=[
                         {'label': 'Global Expansion of Renewables to Achieve the 1.5C Goal', 'value': 'data'},
                ],
            value = "Total Energy Use",
            multi = False,
            clearable = False,
            style = {"width": "50%"}
        )
        ])
])



@app.callback(
    Output(component_id = "graph2", component_property = 'figure'),
    [Input(component_id = "w_dropdown", component_property = 'value')]
)

def update_graph(my_dropdown):
    import plotly.graph_objs as go
    fig = go.Figure()
    colors=['rgb(194,224,239)','rgb(102,153,203)','rgb(174,177,183)','rgb(117,125,143)','rgb(234,128,185)','rgb(204,9,109)']
    fig=go.Figure()

    energys=df['Energy Sector'].unique()
    carriers=df['Carrier Type'].unique()  

    index=0
    plot_data={}
    for energy in energys:
        for carrier in carriers:
            if carrier == 'Electricity':
                plot_data[index]=df[(df['Energy Sector']==energy) & (df['Carrier Type']==carrier)]
                fig.add_trace(go.Scatter(x=plot_data[index]['Year'],
                                        y=plot_data[index]['CumSumEnergy'], 
                                        name=str(energy+' - '+carrier),
                                        mode='none',
                                        line=dict(width=0.5, color=colors[index]),

                                        fillcolor=colors[index],
                                         fill='tonexty',
                                         text=round(plot_data[index]['RE Share']*100,1),
                                         hovertemplate =
                                        '<br><b>'+energy+'</b></b>:<extra></extra>'+
                                        '<br><b>Electricity</b></b>'+
                                        '<br><b>Year</b>: %{x}'+
                                        '<br><b>Demand</b>: %{y}'+
                                        '<br><b>Electrification</b>: %{text}%<br>',
                                        showlegend = False
                                        ))
            else:
                plot_data[index]=df[(df['Energy Sector']==energy) & (df['Carrier Type']==carrier)]
                fig.add_trace(go.Scatter(x=plot_data[index]['Year'],
                                        y=plot_data[index]['CumSumEnergy'],
                                        name=str(energy+' - '+carrier),
                                        mode='none',
                                        line=dict(width=0.5, color=colors[index]),

                                        fillcolor=colors[index],
                                         fill='tonexty',
                                         hovertemplate =
                                        '<br><b>'+energy+'</b></b>:<extra></extra>'+
                                        '<br><b>Year</b>: %{x}'+
                                        '<br><b>Demand</b>: %{y}',
                                        showlegend = False
                                        ))
            index+=1

    fig.update_yaxes(ticktext=['50,000','100,000','150,000','200,000','250,000','300,000','350,000'],
                    tickvals=[50000,100000,150000,200000,250000,300000,350000])

    fig.update_layout(
        width=1000,
        height=800,
        plot_bgcolor='rgba(0,0,0,0)')

    fig.add_shape(type="line",
        x0=2020, y0=0, x1=2020, y1=306000,
        line=dict(
            color="White",
            width=2,
            dash="dot"
        ))

    fig.add_shape(type="line",
        x0=2030, y0=0, x1=2030, y1=284145,
        line=dict(
            color="White",
            width=2,
            dash="dot"
        ))

    fig.add_shape(type="line",
        x0=2040, y0=0, x1=2040, y1=256844,
        line=dict(
            color="White",
            width=2,
            dash="dot"
        ))

    fig.add_trace(go.Scatter(
        x=[2015.4,2015.4,2015.4],
        y=[320000, 195000, 80000],
        mode="text",
        text=["Buildings/Other", 'Industry','Transportation'],
        textposition="top right",
        textfont=dict(
            family="Arial",
            size=18,
            color="white"),
        showlegend = False))

    fig.add_trace(go.Scatter(
        x=[2015.4,2015.4,2015.4,2015.4,2015.4,2015.4],
        y=[310000, 240000, 185000, 117000,70000, 8000],
        mode="text",
        text=["On-site energy", 'Electric power',"On-site energy", 'Electric power',"On-board fuels", 'Electric power'],
        textposition="top right",
        textfont=dict(
            family="Arial",
            size=14,
            color="black"),
        showlegend = False))


    fig.update_layout(
        legend=dict(
            title='Energy Sector - Carrier Type',
            font=dict(
                family="Arial",
                size=12,
                color="black"
            )))

    fig.update_layout(
        title=dict(
            text='<b>Global Eletrification to Achieve the 1.5C Goal</b>',
            font=dict(
                family="Arial",
                size=30,
                color="black"
            )))

    fig.update_layout(
        xaxis_title="Year",
        yaxis_title="Final Energy Demand (PJ)",
        font=dict(
                family="Arial",
                size=18,
                color="black"
        ))

    fig.show()




    return fig


if __name__ == '__main__':
    app.run_server(debug=True, mode='inline',host="127.0.0.1", port=8080)